In [0]:
input_path = "/Volumes/dev/bronze/landing/input/"
schema_loc = "/Volumes/dev/bronze/landing/schema"
checkpoint_loc = "/Volumes/dev/bronze/landing/checkpoint1/"
output_path = "/Volumes/dev/bronze/transactions/autoloader_output"


In [0]:
df_csv = (
    spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "delta")                # Or json, parquet
        .option("cloudFiles.schemaLocation", schema_loc)   # Schema tracking
        .option("cloudFiles.inferColumnTypes", "true")     # Type detection
        .option("cloudFiles.schemaEvolutionMode", "addNewColumns")  
        .option("cloudFiles.allowOverwrites", "true")
        .load(input_path)
)


In [0]:
df_csv.display()
    

In [0]:
df_csv = (
    spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "delta")
        .option("cloudFiles.schemaLocation", schema_loc)
        .option("cloudFiles.inferColumnTypes", "true")
        .option("cloudFiles.schemaEvolutionMode", "addNewColumns")
        .option("cloudFiles.allowOverwrites", "true")
        .load(input_path)
)

query = (
    df_csv.writeStream
        .format("delta")
        .option("checkpointLocation", checkpoint_loc)
        .trigger(availableNow=True)  # Processes all available data and then stops
        .start(output_path)
)

he .trigger(availableNow=True) option in writeStream configures the streaming query to process all available data and then stop automatically. This is useful for batch-like processing of streaming sources, where you want to ingest all current files and terminate the query when done, rather than keeping it running continuously. This mode is supported for Auto Loader and Delta sources in Databricks
1

he .trigger(once=True) option in writeStream configures the streaming query to process all available data in a single micro-batch and then stop automatically. This is useful for scenarios where you want to run streaming logic as a one-time batch job, ingesting all current data and terminating the query when done. It is similar to .trigger(availableNow=True), but processes data in one micro-batch rather than multiple batches if new data arrives during execution.

In [0]:
query = (
    df_csv.writeStream
        .format("delta")
        .option("checkpointLocation", checkpoint_loc)
        .trigger(once=True)
        .start(output_path)
)